In [3]:
import numpy as np
import pandas as pd
import math

entrada='''Me gustas cuando callas porque estás como ausente,
y me oyes desde lejos, y mi voz no te toca.
Parece que los ojos se te hubieran volado
y parece que un beso te cerrara la boca.

Como todas las cosas están llenas de mi alma
emerges de las cosas, llena del alma mía.
Mariposa de sueño, te pareces a mi alma,
y te pareces a la palabra melancolía.

Me gustas cuando callas y estás como distante.
Y estás como quejándote, mariposa en arrullo.
Y me oyes desde lejos, y mi voz no te alcanza:
déjame que me calle con el silencio tuyo.

Déjame que te hable también con tu silencio
claro como una lámpara, simple como un anillo.
Eres como la noche, callada y constelada.
Tu silencio es de estrella, tan lejano y sencillo.

Me gustas cuando callas porque estás como ausente.
Distante y dolorosa como si hubieras muerto.
Una palabra entonces, una sonrisa bastan.
Y estoy alegre, alegre de que no sea cierto.
'''

In [4]:
print(f'\t-----CODIFICACIÓN LZW-----\n')
# Inicializa diccionario estatico, con las letras ordenadas
diccionario_estatico = dict.fromkeys(sorted(entrada))
# inicializo diccionario estatico con los indices y valores 
diccionario_estatico.update(zip(diccionario_estatico,[format(x,'b') for x in range(len(diccionario_estatico))]))
# Inicializo dataframe dinamico -> Copia diccionario estatico
df = pd.DataFrame([[key, int(diccionario_estatico[key],2), diccionario_estatico[key]] for key in diccionario_estatico.keys()], columns=['letra', 'indice_anterior','indice_binario'])
largo_indice= math.ceil(np.log2(len(df)))  # el largo del bit de cada palabra que se va codificando 
indice_dic_dinamico = len(df)
# Acumulador de letras
acumulador = 0
# indice que recorre el texto, la longitud del texto
indice = 0
# Valor que lleva la codificacion
codificacion = ''
# Ciclo que llena el dataframe con la codificacion de los simbolos
while indice + acumulador < len(entrada):
    # LetraCodigo -> Es  la letra o palabra formada para la codificacion del texto, en caso de entrar en el primer if
    letra_codigo =  entrada[indice:indice+acumulador+1]
    # Letras -> Es la letra que se va comprobando si está en el dinamico o si no está para agregarla
    letra = entrada[indice:indice+acumulador+2]
    if letra not in df.values:
        # se suma 0.5 esto por que por ejemplo si el indice es 64, necesita minimo 7 bit para representarse
        largo_indice = math.ceil(np.log2(indice_dic_dinamico+0.5))
        # Codifica el mensaje añadiendo los bits al inicio y la codificacion del indice de la ultima letra o letras 
        dato_codificado = ('{0:0'+str(largo_indice)+'b}').format(df[df['letra']==letra_codigo].index.values[0])
        # Se agrega el dato al dataframe [letra, valor codificado]
        df.loc[len(df)] = [letra, df[df['letra']==letra_codigo].index.values[0],dato_codificado]
        #Indice -> es con el que accedo a los valores del texto, para obtener las palabras de ahi 
        indice += acumulador + 1
        # indiceDiccionarioDinamico el encargado para calcular la longitud de bits de cada palabra-simbolo-letra codificada
        indice_dic_dinamico += 1
        # acumulador  se reinicia para coger empezar a coger al nueva palabra
        acumulador = 0
        # Se suma la codificacion
        codificacion += dato_codificado
    else:
        # el acomulador se aumenta para ir concatenando en caso de que ya se encuentre la palabra en la lista
        acumulador += 1
# Se agrega el ultimo dato
dato_codificado = ('{0:0'+str(largo_indice)+'b}').format(df[df['letra']==letra_codigo].index.values[0])
df.loc[len(df)] = [letra,df[df['letra']==letra_codigo].index.values[0],dato_codificado]
codificacion += dato_codificado
# Se crea un csv con el dataset
df.to_csv('dataset.csv')
print(df)

print(f'\n\t\tmensaje codificado:\n {codificacion}')
f = open ('mensaje_codificado.txt','w')
f.write(codificacion)
f.close()


	-----CODIFICACIÓN LZW-----

     letra  indice_anterior indice_binario
0       \n                0              0
1                         1              1
2        ,                2             10
3        .                3             11
4        :                4            100
..     ...              ...            ...
517    a c              167     0010100111
518    cie              345     0101011001
519    ert              141     0010001101
520  to.\n              479     0111011111
521     \n                0     0000000000

[522 rows x 3 columns]

		mensaje codificado:
 00100001000100000101001001111101110101111000110101110100000100111101111100110101100001000001100111000000110101011001011010111000000101101001100101110000110110011111010100000100010101100010001101011110001111001100100101110110110000110101010110010001001100000111100010001000001000000000100001000000100101110101000001100101000011000011000000100100001000011101101100000010010110001000100101010011001001110100000

In [ ]:
print(8*len(entrada)/len(codificacion))
8/1.78

2.006779661016949


4.49438202247191

In [ ]:
from operator import le


print(f'\t-----DECODIFICACIÓN LZW-----\n')

diccionario_estatico = dict.fromkeys(sorted(entrada))
diccionario_estatico.update(zip(diccionario_estatico,[format(x,'b') for x in range(len(diccionario_estatico))]))
largo_indice= math.ceil(np.log2(len(diccionario_estatico)))  # el largo del bit de cada palabra que se va codificando 
indice_dic_dinamico = len(diccionario_estatico)
df = pd.DataFrame([[key, int(diccionario_estatico[key],2), ('{:0{}b}').format(int(diccionario_estatico[key],2), largo_indice)] for key in diccionario_estatico.keys()], columns=['letra', 'indice_anterior','indice_binario'])
print(largo_indice)
indice_letra = '0'
indice = 0
indice2 = largo_indice
letra = ''
'''while indice < len(codificacion):
    bits = codificacion[indice:indice2]
    print(f'bits: {bits}')
    print(f'indice: {indice}')
    print(f'indice2: {indice2}')
    print(f'Resta: {indice2-largo_indice}')
    bits_corte = bits[-largo_indice:]
    if(bits_corte in df.values):
        letra += df[df['indice_binario']==bits_corte].letra.values[0]
    print(f'bits cortantes: {bits_corte}')
    print(f'letra: {letra}')
    if(letra not in df.values):
        indice_letra = df[df['indice_binario']==bits[-largo_indice:]].indice_anterior.values[0]
        df.loc[len(df)] = [letra, indice_letra, bits[-largo_indice:]]
        indice_dic_dinamico += 1
        letra = ''
        indice = indice2-largo_indice
        indice2 = indice+largo_indice
        print(df)
    else:
        indice2 += largo_indice
    largo_indice = math.ceil(np.log2(indice_dic_dinamico+0.5))'''
    
    
    
        



'''msj = ''

while indice < len(codificacion):
    largo_indice = math.ceil(np.log2(indice_dic_dinamico+0.5))
    bits = codificacion[indice:indice+largo_indice]
    indice += largo_indice
    indice_dic_dinamico += 1
    indice_letra = df[df['indice_binario']==bits].indice_anterior.values[0]
    msj += df.letra[indice_letra]'''



	-----DECODIFICACIÓN LZW-----

   letra  indice_anterior indice_binario
0      _                0          00000
1      a                1          00001
2      b                2          00010
3      c                3          00011
4      d                4          00100
5      e                5          00101
6      i                6          00110
7      l                7          00111
8      m                8          01000
9      n                9          01001
10     o               10          01010
11     r               11          01011
12     s               12          01100
13     t               13          01101
14     u               14          01110
15     v               15          01111
16     y               16          10000
bits: 00001
indice: 0
indice2: 5
Resta: 0
bits cortantes: 00001
letra: 
bits: 0000100000
indice: 0
indice2: 10
Resta: 5
bits cortantes: 00000
letra: a
bits: 000010000000011
indice: 0
indice2: 15
Resta: 10
bits cortantes: 00011
letr

IndexError: index 0 is out of bounds for axis 0 with size 0